In [3]:
import cv2
import numpy as np


# 1. 영상의 필터링

## 엠보싱 필터링

In [7]:
src= cv2.imread("preview.jpg", cv2.IMREAD_GRAYSCALE)

if src is None:
    print("failed")
    exit()

emboss = np.array([[-1,-1,0],
                    [-1,0,1],
                    [0,1,1]], np.float32)
dst = cv2.filter2D(src, -1, emboss, delta=128)

cv2.imshow('src',src)
cv2.imshow('dst',dst)

cv2.waitKey()
cv2.destroyAllWindows()

## 2. 블러링 : 영상부드럽게 하기


- blurring_mean() 함수는 3×3, 5×5, 7×7 크기의 평균값 필 터를 이용하여 rose.bmp 장미 영상을 부드럽게 변환하고 그 결과를 화면에 출력
- 평균값필터의크기가커질수록결과영상이더욱부드럽 게 변경되는 것을 확인할 수 있음

In [10]:
def blurring_mean():
    src = cv2.imread("preview.jpg",cv2.IMREAD_GRAYSCALE)

    if src is None:
        print("failed")
        exit()

    cv2.imshow("src",src)

    for ksize in range(3,9,2):
        dst = cv2.blur(src, (ksize, ksize))

        desc = "Mean: %d %d" %(ksize, ksize)
        cv2.putText(dst, desc, (10,30), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                    1.0, 255, 1, cv2.LINE_AA)
        
        cv2.imshow('dst',dst)
        cv2.waitKey()

    cv2.destroyAllWindows()

blurring_mean()

## 가우시안 필터
- blurring_gaussian() 함수는 가우시안 표준 편차를 1부터 5 까지 정수 단위로 증가시키면서 rose.bmp 장미 영상에 대 해 가우시안 필터링을 수행
- src는입력영상인rose.bmp파일이고,dst는 GaussianBlur() 함수에 의해 생성된 블러링 결과 영상
- 표준편차값이커질수록결과영상이더욱부드럽게변경 되는 것을 확인

In [12]:
def blurring_gaussian():
    src = cv2.imread("preview.jpg", cv2.IMREAD_GRAYSCALE)
    if src is None:
        print("failed")
        return

    cv2.imshow("src",src)

    for sigma in range(1,6):
        dst = cv2.GaussianBlur(src,(0,0),sigma)
        desc = "Gaussian: sigma= %d" %(sigma)
        cv2.putText(dst, desc, (10,30), cv2.FONT_HERSHEY_SIMPLEX,
                    1.0, 255, 1, cv2.LINE_AA)
        cv2.imshow("dst",dst)
        cv2.waitKey()
    cv2.destroyAllWindows()

blurring_gaussian()
    

# 3. 샤프닝 
## 언샤프 마스크 필터

- 코드는 rose.bmp 장미 영상을 다양한 표준 편차 값으로 가 우시안 필터를 적용하고, 블러링된 영상을 이용하여 샤프닝 결과 영상을 생성
- src창에나타난장미영상은입력영상이고,dst창에나타 난 영상은 다양한 sigma 값에 의해 생성된 언샤프 마스크 필터링 결과
- dst영상이장미꽃경계부분이좀더뚜렷하게구분이되는 것을 확인할 수 있음
- 다만sigma값이커짐에따라다소과장된느낌의샤프닝 결과 영상이 만들어질 수도 있으니 주의

# 4. 잡음
## 가우시안 잡음 모델
- noise_gaussian() 함수는 레나 영상에 평균이 0이고 표준 편차가 각각 10, 20, 30인 가우시안 잡음을 추가하여 화면 에 나타냄
- 원본영상src에비해가우시안잡음이추가된결과영상 dst가 거칠고 지저분해 보이는 것을 확인할 수 있음
- 특히 표준 편차 stddev 값이 증가함에 따라 잡음의 영향이 커지므로 결과 영상이 더욱 지저분해 지는 것을 볼 수 있음

In [16]:
def noise_gaussian():
    src = cv2.imread("preview.jpg", cv2.IMREAD_GRAYSCALE)
    
    if src is None:
        print("failed")
        return
    
    cv2.imshow('src',src)

    for stddev in [10,20,30]:
        noise = np.zeros(src.shape, np.int32)
        cv2.randn(noise, 0, stddev)

        dst = cv2.add(src, noise, dtype=cv2.CV_8UC1)

        desc = "stddev = %d" %stddev

        cv2.putText(dst, desc, (10,30), cv2.FONT_HERSHEY_SIMPLEX,
                    1.0, 225, 1, cv2.LINE_AA)
        cv2.imshow('dst',dst)

        cv2.waitKey()
    cv2.destroyAllWindows()

noise_gaussian()

## 양방향 필터


- src 창의 영상은 lenna.bmp 영상에 평균이 0이고 표준 편차
가 5인 가우시안 잡음이 추가된 영상
-  이영상에대해표준편차가5인가우시안필터링을수행한
결과가 dst1 영상
- 입력영상src에비해지글거리는잡음의영향은크게줄었 지만, 머리카락, 모자, 배경 사물의 경계 부분이 함께 블러 링되어 흐릿하게 변경됨
- 반면에 양방향 필터가 적용된 src2 영상은 머리카락, 모자, 배경 사물의 경계는 그대로 유지됨

cv2.GaussianBlur(src, ksize, sigmaX)

- src: 입력 이미지.
- ksize: 커널의 크기를 나타내는 튜플. (0, 0)이면 자동으로 커널 크기가 계산됩니다. 커널 크기가 커질수록 블러링 효과가 더 강화됩니다.
- sigmaX: X 방향으로의 가우시안 커널 표준 편차. 이 값이 커질수록 블러링이 강화됩니다.


cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace)

- src: 입력 이미지.
- d: 필터링에 사용되는 지름(diameter)입니다. 이 지름 내에서 픽셀 값만 사용됩니다.
- sigmaColor: 색상 공간에서의 필터링 시 가우시안 함수의 표준 편차입니다. 이 값이 크면 더 넓은 범위의 색상 값이 포함됩니다.
- sigmaSpace: 공간 공간에서의 필터링 시 가우시안 함수의 표준 편차입니다. 이 값이 크면 더 넓은 공간적 범위가 포함됩니다.

cv2.GaussianBlur 함수는 이미지를 블러링(흐리게) 처리하는 데 사용되며, sigmaX를 조절하여 블러링 정도를 조절할 수 있습니다.  

cv2.bilateralFilter 함수는 이미지의 선명도를 유지하면서 노이즈를 감소시키는데 사용됩니다. d, sigmaColor, 및 sigmaSpace 파라미터를 조절하여 필터링의 효과를 조절할 수 있습니다.

In [17]:
def filter_bilateral():
    src = cv2.imread('preview.jpg', cv2.IMREAD_GRAYSCALE)

    if src is None:
        print("failed")
        return 
    
    noise = np.zeros(src.shape, np.int32)
    cv2.randn(noise, 0 ,5)
    cv2.add(src, noise, src, dtype=cv2.CV_8UC1)

    dst1 = cv2.GaussianBlur(src, (0,0),5)
    dst2 = cv2.bilateralFilter(src, -1, 10,5)
    
    cv2.imshow("src", src)
    cv2.imshow("dst1",dst1)
    cv2.imshow("dst2",dst2)
    cv2.waitKey()
    cv2.destroyAllWindows()

filter_bilateral()


## 미디언 필터


- filter_median() 함수는 입력 영상 전체 크기의 10%에 해당 하는 픽셀에 소금&후추 잡음을 추가
- 가우시안 필터와 미디언 필터를 수행한 결과 영상을 화면에 출력
- 소금&후추 잡음에 대해서는 가우시안 블러링을 적용하여 도 여전히 영상이 지저분하게 보이는 것을 확인할 수 있음
- 반면에 미디언 필터를 적용한 dst2 영상에서는 잡음에 의해 추가된 흰색 또는 검은색 픽셀이 효과적으로 제거된 것을 확인할 수 있음

In [23]:
import random

def filter_median():
    src = cv2.imread("preview.jpg",cv2.IMREAD_GRAYSCALE)

    if src is None:
        print("failed")
        return
    
    for i in range(0, int(src.size / 10)):
        x = random.randint(0, src.shape[1]-1)
        y = random.randint(0, src.shape[0]-1)
           # 무작위로 선택한 좌표가 이미지 범위를 벗어나는 경우 다시 선택
        # while x >= src.shape[1] or y >= src.shape[0]:
        #     x = random.randint(0, src.shape[1] - 1)
        #     y = random.randint(0, src.shape[0] - 1)
        src[y,x] = (i % 2) * 255


    dst1 = cv2.GaussianBlur(src, (0,0), 1)
    dst2 = cv2.medianBlur(src, 3)

    cv2.imshow("src", src)
    cv2.imshow("dst1", dst1)
    cv2.imshow("dst2", dst2)
    cv2.waitKey()
    cv2.destroyAllWindows()

filter_median()